In [1]:
import pandas as pd

pd.set_option('display.max_rows', 50) # показывать больше строк
pd.set_option('display.max_columns', 50) # показывать больше колонок

# Data import

In [2]:
df_odd = pd.read_excel('C:/Users/Igor/Downloads/even/nl-odd-weekend.ods', engine="odf")
df_even = pd.read_excel('C:/Users/Igor/Downloads/even/nl-even-weekend.ods', engine="odf")

# Functions (Функции)

## Preparation

In [3]:
def name_columns(df):
    df.columns = ['empty_0','line','route','train','empty_1','station','short_1','short_2','arrival','departure',
                  'empty_2','empty_3','train_action','empty_4','passengers']

## Work with time

### Filling time gaps.

In [4]:
def df_time_filler(df, t):
    df['arrival'] = df['arrival'].astype(str).apply(lambda x: None if x.strip() == '' else x)
    df['arrival'] = df['arrival'].astype(str).apply(lambda x: '{0}:{0}:{0}'.format(t) if x.strip() == 'nan' else x)
    df['departure'] = df['departure'].astype(str).apply(lambda x: None if x.strip() == '' else x)
    df['departure'] = df['departure'].astype(str).apply(lambda x: '{0}:{0}:{0}'.format(t) if x.strip() == 'nan' else x)
    df['arrival'] = pd.to_datetime(df['arrival'])
    df['departure'] = pd.to_datetime(df['departure'])
    print('row count before cut is: ', df.shape[0])

### Cutting the schedule

In [5]:
# the new schedule starts at 4 a.m.
# the morning schedule (4a.m. - morning_hour_cut)
def df_morning(df, morning_hour_cut):
    df.drop(df[((df['arrival'].dt.hour >= 0) & (df['arrival'].dt.hour < 4) | 
                (df['arrival'].dt.hour >= morning_hour_cut) & (df['arrival'].dt.hour <= 23))].index, inplace=True)
    df.drop(df[((df['departure'].dt.hour >= 0) & (df['departure'].dt.hour < 4) | 
                (df['departure'].dt.hour >= morning_hour_cut) & (df['departure'].dt.hour <= 23))].index, inplace=True)
    print('row count after cut is: ', df.shape[0])
    print('---------------------------')
    
# the morning_day schedule (4a.m. - night_hour_cut)    
def df_morning_day(df, night_hour_cut):
    df.drop(df[(df['arrival'].dt.hour >= night_hour_cut) & 
               (df['arrival'].dt.hour < 4)].index, inplace=True)
    df.drop(df[(df['departure'].dt.hour >= night_hour_cut) & 
               (df['departure'].dt.hour < 4)].index, inplace=True)
    print('row count after cut is: ', df.shape[0])
    print('---------------------------')
    
# the day_night schedule (morning_hour_cut - 4a.m.)    
def df_day_night(df, morning_hour_cut):
    df.drop(df[(df['arrival'].dt.hour < morning_hour_cut) & 
               (df['arrival'].dt.hour > 4)].index, inplace=True)
    df.drop(df[(df['departure'].dt.hour < morning_hour_cut) & 
               (df['departure'].dt.hour > 4)].index, inplace=True)
    print('row count after cut is: ', df.shape[0])
    print('---------------------------')
    
# the night schedule (night_hour_cut - 4a.m.)    
def df_night(df, night_hour_cut):
    df.drop(df[((df['arrival'].dt.hour >= 0) & (df['arrival'].dt.hour < night_hour_cut) | 
                (df['arrival'].dt.hour >= 4) & (df['arrival'].dt.hour <= 23))].index, inplace=True)
    df.drop(df[((df['departure'].dt.hour >= 0) & (df['departure'].dt.hour < night_hour_cut) | 
                (df['departure'].dt.hour >= 4) & (df['departure'].dt.hour <= 23))].index, inplace=True)
    print('row count after cut is: ', df.shape[0])
    print('---------------------------')

### Delete time at start and end points

In [6]:
def clean_time(df):
    df['arrival'] = [time.time() for time in df['arrival']]
    df['departure'] = [time.time() for time in df['departure']]
    df.loc[(df['train_action'] == 4) | 
           (df['train_action'] == 6) | 
           (df['train_action'] == 1), 'arrival'] = ''
    df.loc[(df['train_action'] == 3) | 
           (df['train_action'] == 5) | 
           (df['train_action'] == 2), 'departure'] = ''
    print('clean time: Done')
    print('---------------------------')

## Errors

In [7]:
def arrival_departure_error(df):
    print('arrival-departure error check:')
    df_arrival_departure_error = df.loc[(df['arrival'].dt.time >= df['departure'].dt.time)]
    df_arrival_departure_error = df_arrival_departure_error.loc[:, ['route', 'train', 'station', 'arrival', 
                                                                'departure','train_action','passengers']]
    print('row count is: ', df_arrival_departure_error.shape[0]) 
    df_arrival_departure_error.drop(df_arrival_departure_error[(df_arrival_departure_error['arrival'].dt.time == 
                                                            df_arrival_departure_error['departure'].dt.time) & 
                                                           (df_arrival_departure_error['passengers'] == 1)].index, inplace=True)
    print('row count without empty trains is: ', df_arrival_departure_error.shape[0])
    print('row count without reversal is: ', (df_arrival_departure_error.train.value_counts()>2).sum())
    print('---------------------------')
    
def start_end_points_check(df):
    print('start_end points check:')
    df_route = df.drop_duplicates(['route'], keep='last')
    df_start_point = df[df['train_action'] == 1]
    df_end_point = df[df['train_action'] == 2]
    df_start_Uturn = df[(df['train_action'] == 4) | (df['train_action'] == 6)]
    df_end_Uturn = df[(df['train_action'] == 3) | (df['train_action'] == 5)]
    print('route count is: ' + str(df_route.shape[0])) 
    print('start points count is: ' + str(df_start_point.shape[0]))
    print('end points count is: ' + str(df_end_point.shape[0]))
    print('start U-turn points count is: ' + str(df_start_Uturn.shape[0]))
    print('end U-turn points count is: ' + str(df_end_Uturn.shape[0]))
    print('---------------------------')
    
def errors_check(df):
    arrival_departure_error(df)
    clean_time(df)
    start_end_points_check(df)

## Other functions

In [8]:
def short_filler(df, station, short):
    df.loc[df['station'] == station, ('short_1', 'short_2')] = short
    print('row with station = {0} filled in short_1 and short_2 by {1}'.format(station, short))
    print('---------------------------')
    
def line_revers(df):
    df['line'] = (df['line'] == 2).astype(int)
    df.loc[df['line'] == 0, 'line'] = 2
    
def exporter(df, line_name, constructor_pattern, day_of_week):
    df.drop(['empty_0'], inplace = True, axis = 1)
    df.to_excel("{0}_{1}_{2}.xlsx".format(line_name, constructor_pattern, day_of_week))
    print('data exported to file {0}_{1}_{2}.xlsx'.format(line_name, constructor_pattern, day_of_week))

## Assembling scripts

In [9]:
def odd_even_assembling():
    constructor_pattern = 'odd_even'    
    name_columns(df_odd)
    name_columns(df_even)
    
    #working whit time data and cutting the schedule for odd days of the month
    #odd
    df_time_filler(df_odd, '13')
    df_morning_day(df_odd, night_hour_cut)
    #even
    df_time_filler(df_even, '01')
    df_night(df_even, night_hour_cut)
    
    #some schedules do not have short station names. the function fills in gaps
    short_filler(df_odd, station, short)
    short_filler(df_even, station, short)
    #for 'nl' change the direction of movement in the 'line' column
    if line_name == 'nl':
        line_revers(df_odd)
        line_revers(df_even)
        print("revers lines for 'nl' only")
        print('---------------------------')
    
    #combining schedules
    df_odd_even = pd.concat([df_odd, df_even])
    
    errors_check(df_odd_even)
    
    exporter(df_odd_even, line_name, constructor_pattern, day_of_week)
    
def even_odd_assembling():
    constructor_pattern = 'even_odd'
    name_columns(df_odd)
    name_columns(df_even)
    
    #working whit time data and cutting the schedule for odd days of the month
    #odd
    df_time_filler(df_odd, '13')
    df_day_night(df_odd, morning_hour_cut)
    #even
    df_time_filler(df_even, '05')
    df_morning(df_even, morning_hour_cut)
    
    #some schedules do not have short station names. the function fills in gaps
    short_filler(df_odd, station, short)
    short_filler(df_even, station, short)
    #for 'nl' change the direction of movement in the 'line' column
    if line_name == 'nl':
        line_revers(df_odd)
        line_revers(df_even)
        print("revers lines for 'nl' only")
        print('---------------------------')
        
    #combining schedules
    df_even_odd = pd.concat([df_even, df_odd])
    
    errors_check(df_even_odd)
    
    exporter(df_even_odd, line_name, constructor_pattern, day_of_week)

# Data input

In [10]:
morning_hour_cut = 7
night_hour_cut = 1
station = 'Электрозаводская'
short = 'rbc'
line_name = 'nl'
day_of_week = 'weekend'
global df_odd_even

# Script

In [11]:
odd_even_assembling()

row count before cut is:  6049
row count after cut is:  5937
---------------------------
row count before cut is:  6029
row count after cut is:  112
---------------------------
row with station = Электрозаводская filled in short_1 and short_2 by rbc
---------------------------
row with station = Электрозаводская filled in short_1 and short_2 by rbc
---------------------------
revers lines for 'nl' only
---------------------------
arrival-departure error check:
row count is:  611
row count without empty trains is:  565
row count without reversal is:  0
---------------------------
clean time: Done
---------------------------
start_end points check:
route count is: 19
start points count is: 19
end points count is: 19
start U-turn points count is: 535
end U-turn points count is: 535
---------------------------
data exported to file nl_odd_even_weekend.xlsx
